# 安装必须要包

In [8]:
#### install third party libs
#!pip install sqlalchemy
#!pip install mysql-connector-python
!pip install pymysql
#!pip install sqlalchemy
!pip install DBUtils

import os
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

import tqdm
from tqdm import tqdm
tqdm.pandas(desc='apply')
import pymysql


# 本地mysql数据库连接

In [13]:
import pymysql

## 尝试关闭mysql connection
def get_mysql_connection():
    ''' try:
        conn.close()
    except Exception as e:
        print(e) 
    finally:
        print('mysql connection closed!')
        pass
    '''
    
    conn = pymysql.connect(
        host='localhost',
        port=int(3306),
        user="root",
        passwd='0921051011Liu',
        db="faers",
        charset='utf8mb4')
    
    return conn

conn = get_mysql_connection()

## mysql 数据库连接池

In [3]:
!pip install DButils==1.2
import DBUtils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 134.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for DButils: filename=DBUtils-1.2-py3-none-any.whl size=126476 sha256=4d36e7d61f7194dd799e2cdb8f5ae134a73cbab2119953dd657cd0229211ae79
  Stored in directory: /Users/liuhaoran/Library/Caches/pip/wheels/c9/d7/41/92c5b5a10fb27b51b2b7437822390fcc8fe8ef1250fe0b4601
Successfully built DButils
  Attempting uninstall: DButils
    Found existing installation: DBUtils 3.0.3
    Uninstalling DBUtils-3.0.3:
      Successfully uninstalled DBUtils-3.0.3


In [5]:
import pymysql
import DBUtils
from timeit import default_timer
from DBUtils.PooledDB import PooledDB
# https://zhuanlan.zhihu.com/p/687472604

class DMysqlConfig:
    """

        :param mincached:连接池中空闲连接的初始数量
        :param maxcached:连接池中空闲连接的最大数量
        :param maxshared:共享连接的最大数量
        :param maxconnections:创建连接池的最大数量
        :param blocking:超过最大连接数量时候的表现，为True等待连接数量下降，为false直接报错处理
        :param maxusage:单个连接的最大重复使用次数
        :param setsession:optional list of SQL commands that may serve to prepare
            the session, e.g. ["set datestyle to ...", "set time zone ..."]
        :param reset:how connections should be reset when returned to the pool
            (False or None to rollback transcations started with begin(),
            True to always issue a rollback for safety's sake)
        :param host:数据库ip地址
        :param port:数据库端口
        :param db:库名
        :param user:用户名
        :param passwd:密码
        :param charset:字符编码
    """

    def __init__(self, host, db, user, password, port=3306):
        self.host = host
        self.port = port
        self.db = db
        self.user = user
        self.password = password

        #self.charset = 'UTF8'  # 不能是 utf-8
        self.minCached = 10
        self.maxCached = 20
        self.maxShared = 10
        self.maxConnection = 100

        self.blocking = True
        self.maxUsage = 100
        self.setSession = None
        self.reset = True


# ---- 用连接池来返回数据库连接
class DMysqlPoolConn:

    __pool = None

    def __init__(self, config):

        if not self.__pool:
            self.__class__.__pool = PooledDB(creator=pymysql,
                                             maxconnections=config.maxConnection,
                                             mincached=config.minCached,
                                             maxcached=config.maxCached,
                                             maxshared=config.maxShared,
                                             blocking=config.blocking,
                                             maxusage=config.maxUsage,
                                             setsession=config.setSession,
                                             #charset=config.charset,
                                             host=config.host,
                                             port=config.port,
                                             database=config.db,
                                             user=config.user,
                                             password=config.password,
                                             )

    def get_conn(self):
        return self.__pool.connection()


# ========== 在程序的开始初始化一个连接池
host = 'localhost'
port = 3306
db = 'faers'
user = 'root'
password = '0921051011Liu'
db_config = DMysqlConfig(host, db, user, password, port)
g_pool_connection = DMysqlPoolConn(db_config)
conn = g_pool_connection.get_conn()

# 基础数据检查 

## 对demo表进行去重, 
sql 如下：
`DROP TABLE if exists faers.demo_dedup;
CREATE TABLE faers.demo_dedup as
SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY CASEID ORDER BY FDA_DT DESC, PRIMARYID DESC) as rn
    FROM faers.faers_demo
;`

In [ ]:
# https://www.nature.com/articles/s41598-024-58460-6
# and then PRIMARYID. For reports with the same CASEID, we retain the one with the largest FDA_DT value. Secondly, for reports where both CASEID and FDA_DT are the same,
# see faers.demo_dedup

In [6]:
import pandas as pd
df = pd.read_sql_query("select rn,count(1),count(distinct primaryid),count(distinct caseid) from faers.demo_dedup group by rn order by rn;", conn)
df

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/136583447.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select rn,count(1),count(distinct primaryid),count(distinct caseid) from faers.demo_dedup group by rn order by rn;", conn)


,rn,count(1),count(distinct primaryid),count(distinct caseid)
0,1,10219652,10219652,10219652
1,2,1225829,1225829,1225829
2,3,262514,262514,262514
3,4,90825,90825,90825
4,5,38521,38521,38521
5,6,18732,18732,18732
6,7,9734,9734,9734
7,8,5281,5281,5281
8,9,2858,2858,2858
9,10,1557,1557,1557


In [12]:
df = pd.read_sql_query("select count(1),count(distinct primaryid),count(distinct caseid) from faers.faers_drug;", conn)
df

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/4200782247.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("select count(1),count(distinct primaryid),count(distinct caseid) from faers.faers_drug;", conn)


,count(1),count(distinct primaryid),count(distinct caseid)
0,17223552,4362113,3895924


In [42]:
sql = ''' 
--   获得药品相关的所有case
DROP TABLE  if exists faers.ribo_drug_usage;
CREATE TABLE faers.ribo_drug_usage as
select *
from faers.faers_drug d1
where (lower(d1.drugname) like 'ribociclib%'
        OR lower(d1.drugname) like '%kisqali%'
        OR lower(d1.prod_ai) like '%ribociclib%'
        OR lower(d1.prod_ai) like '%kisqali%'
    ) and lower(d1.role_cod) = 'ps'
    ;

select count(1),count(distinct primaryid),count(distinct caseid) from faers.ribo_drug_usage;
 '''

df = pd.read_sql_query(sql, conn)
df.tail(10)

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_8244/2129124748.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


DatabaseError: Execution failed on sql ' 
-- 获得药品相关的所有case
DROP TABLE  if exists faers.ribo_caseid_tb;
CREATE TABLE faers.ribo_caseid_tb as
select primaryid, caseid, DRUG_SEQ,START_DT
from faers.faers_drug d1
where (lower(d1.drugname) like 'ribociclib%'
        OR lower(d1.drugname) like '%kisqali%'
        OR lower(d1.prod_ai) like '%ribociclib%'
        OR lower(d1.prod_ai) like '%kisqali%'
    ) and lower(d1.role_cod) = 'ps'
; ': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CREATE TABLE faers.ribo_caseid_tb as\nselect primaryid, caseid, DRUG_SEQ,START_DT' at line 3")

In [39]:
sql = ''' 
    -- DROP TABLE faers.ribo_related_caseid_primary_tb;
    CREATE TABLE faers.ribo_related_caseid_primary_tb as
    select  d1.caseid, d1.primaryid
    from faers.faers_drug d1
    where (lower(d1.drugname) like 'ribociclib%'
            OR lower(d1.drugname) like '%kisqali%'
            OR lower(d1.prod_ai) like '%ribociclib%'
            OR lower(d1.prod_ai) like '%kisqali%'
        ) and caseid is not null and caseid not in ('', '0')
        and primaryid is not null and primaryid not in ('', '0')
    group by 1,2;
    '''

df = pd.read_sql_query(sql, conn)
df.tail(10)

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_61625/223975595.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


TypeError: 'NoneType' object is not iterable

In [ ]:

sql = '''
SELECT  sex, COUNT(distinct tb1.caseid) AS caseid_count
FROM
(   select  primaryid, sex
    from faers.demo_dedup
    where rn = 1
) tb1
join (
        select distinct caseid
        from faers.ribo_related_caseid_primary_tb
) tb2
on tb1.caseid = tb2.caseid
group by sex;
'''

## 比较 faers.ribo_drug_usage_non_ps 和 faers.ribo_drug_usage

In [16]:
sql = ''' 
    select  t1.caseid as caseid, t1.primaryid, 
            t2.primaryid as non_ps_primaryid, t2.role_cod as non_ps_role_cod
    from (
    select caseid,primaryid
    from faers.ribo_drug_usage
    group by caseid,primaryid
    ) t1
    join 
    (
        select caseid,primaryid,role_cod
        from faers.ribo_drug_usage_non_ps
        group by caseid,primaryid,role_cod
    ) t2
    on t1.caseid = t2.caseid 
    '''
# 结论：存在这样的caseid:同时包含 PS 和 非PS作用
df = pd.read_sql_query(sql, conn)
df.tail(10)

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_61625/2390358115.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


,caseid,primaryid,non_ps_primaryid,non_ps_role_cod
17229,23513272,235132721,235132721,SS
17230,23519209,235192091,235192091,SS
17231,23530507,235305072,235305072,SS
17232,23560238,235602384,235602384,SS
17233,23566852,235668522,235668522,SS
17234,23637234,236372341,236372341,SS
17235,23649427,236494271,236494271,SS
17236,23659698,236596981,236596981,SS
17237,23673471,236734712,236734712,SS
17238,23681982,236819821,236819821,SS


# Table1.1性别

In [ ]:
## select * from faers_reac limit

In [7]:
sql = '''

SELECT  
    case 
        when sex in ('UNK','') then  'Unknown'
        when sex = 'M' then  'Male'
        when sex = 'F' then  'Female'
        else sex
    end as sex, COUNT(distinct tb1.primaryid) AS primaryid_count
FROM
(   select  primaryid,  sex
    from faers.demo_dedup
    where rn = 1
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid
group by 1 order by 1;

'''
df_sex = pd.read_sql_query(sql, conn)
df_sex


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/1246684853.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sex = pd.read_sql_query(sql, conn)


,sex,primaryid_count
0,Female,13932
1,Male,269
2,Unknown,1124


# Table1.2年龄


In [8]:
sql = '''

select
    case
            when age = '无' or cast(age as signed) < 0 then 'Unknown'
            when cast(age as signed) >= 0  and cast(age as signed) < 18 then '＜18'
            when cast(age as signed) >= 18 and cast(age as signed) < 29 then '18-29'
            when cast(age as signed) >= 30 and cast(age as signed) < 49 then '30-49'
            when cast(age as signed) >= 50 and cast(age as signed) < 64 then '50-64'
            when cast(age as signed) >= 65 and cast(age as signed) < 74 then '65-74'
            when cast(age as signed) > 75  and cast(age as signed) < 84 then '75-84'
            when cast(age as signed) > 75  and cast(age as signed) < 84 then '75-84'
            else '>=85'
    end as age_group,
    COUNT(distinct tb1.primaryid) AS primaryid_count
FROM
(
    select primaryid, age
    from faers.demo_dedup
    where rn = 1
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid
group by 1
order by 1
;
'''
df_age = pd.read_sql_query(sql, conn)
df_age


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/1584813064.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_age = pd.read_sql_query(sql, conn)


,age,primaryid_count
0,＜18,19
1,>=85,803
2,18-29,39
3,30-49,1391
4,50-64,2307
5,65-74,1663
6,75-84,756
7,Unknown,8347


## 年龄的分位


In [9]:
sql = '''

select 
    percentile_cont(0.5) within group (order by age) as IQR
FROM
(
    select primaryid, age
    from faers.demo_dedup
    where age not in ('Unknown','无') and cast(age as signed) >= 0
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid;

'''

df_age_q = pd.read_sql_query(sql, conn)
df_age_q


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/282076065.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_age_q = pd.read_sql_query(sql, conn)


DatabaseError: Execution failed on sql '

select 
    percentile_cont(0.5) within group (order by age) as IQR
FROM
(
    select primaryid, age
    from faers.faers_demo
    where age not in ('Unknown','无') and cast(age as signed) >= 0
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid;

': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '(order by age) as IQR\nFROM\n(\n    select primaryid, age\n    from faers.faers_demo' at line 2")

# Table1.3体重


## 体重的分桶计


In [13]:
sql = '''

select
    case
            when wt in ('无','wt_convert_fail') then 'Unknown'
            when cast(wt as signed) >= 0  and cast(wt as signed) < 80 then '＜80'
            when cast(wt as signed) >= 80 and cast(wt as signed) < 100 then '80-100'
            when cast(wt as signed) >= 100 then '>100'
            else wt
    end as wt_bucket,
    COUNT(distinct tb1.primaryid) AS primaryid_count
FROM
(
    select primaryid, wt
    from faers.demo_dedup
    where rn = 1
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid
group by 1
order by 1
;
'''
df_wt = pd.read_sql_query(sql, conn)
df_wt


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/1127711843.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_wt = pd.read_sql_query(sql, conn)


,wt_bucket,primaryid_count
0,＜80,2231
1,>100,168
2,80-100,567
3,Unknown,12359


## 体重的分位点

In [100]:
sql = '''

select 
    percentile(wt, 0.25) as wt_q25,
    percentile(wt, 0.5)  as wt_q50,
    percentile(wt, 0.75) as wt_q75
FROM
(
    select caseid, max(wt) as wt
    from faers.faers_demo
    where upper(I_F_COD) = 'I'   
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid;

'''

df_wt_q = pd.read_sql_query(sql, conn)
df_wt_q


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_24540/2231694097.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_wt_q = pd.read_sql_query(sql, conn)


DatabaseError: Execution failed on sql '

select 
    percentile(wt, 0.25) as wt_q25,
    percentile(wt, 0.5)  as wt_q50,
    percentile(wt, 0.75) as wt_q75
FROM
(
    select caseid, max(wt) as wt
    from faers.faers_demo
    where upper(I_F_COD) = 'I'   
    group by caseid
) tb1
join (
        select distinct caseid
        from faers.ribo_drug_usage
) tb2
on tb1.caseid = tb2.caseid;

': (1305, 'FUNCTION faers.percentile does not exist')

# Table1.4上报者职业

In [14]:
sql = '''

select
    OCCP_COD,
    COUNT(distinct tb1.primaryid) AS primaryid_count
FROM
(
    select primaryid, OCCP_COD
    from faers.demo_dedup
    where rn = 1
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid
group by 1
order by 1
;
'''
df_reporter = pd.read_sql_query(sql, conn)
df_reporter



/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/3694848735.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reporter = pd.read_sql_query(sql, conn)


,OCCP_COD,primaryid_count
0,,316
1,CN,8115
2,HP,1701
3,MD,3835
4,OT,792
5,PH,566


# Table1.5不良反应造成的结局

In [15]:
sql = '''

select
    OUTC_COD,
    COUNT(distinct tb1.primaryid) AS primaryid_count
FROM
(
    select primaryid, max(OUTC_COD) as OUTC_COD
    from faers.faers_outc
    group by primaryid
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid
join (
    select primaryid
    from faers.demo_dedup
    where rn = 1
) tb3 on tb2.primaryid = tb3.primaryid
group by 1
order by 1
;
'''
df_outcome = pd.read_sql_query(sql, conn)
df_outcome



/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/4220415200.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outcome = pd.read_sql_query(sql, conn)


,OUTC_COD,primaryid_count
0,0,2036
1,1,81
2,2,1972
3,3,67
4,4,1
5,5,15
6,6,7105


In [31]:
df_outcome


,OUTC_COD,caseid_count
0,0,2034
1,1,82
2,2,1973
3,3,66
4,4,1
5,5,15
6,6,7270


# Table1.6不良反应上报的国家

,primaryid_count
continent,
Africa,339
Asia/Japan,1749
Europe,3050
Missing,2101
North America,6371
Oceania,183
South America,1532


In [16]:
sql = '''

select
    REPORTER_COUNTRY,
    COUNT(distinct tb1.primaryid) AS primaryid_count
FROM
(
    select primaryid, REPORTER_COUNTRY
    from faers.demo_dedup
    where rn = 1
) tb1
join (
        select distinct primaryid
        from faers.ribo_drug_usage
) tb2
on tb1.primaryid = tb2.primaryid
group by 1
order by 1
;
'''
df_reporter_country = pd.read_sql_query(sql, conn)



/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/2444512671.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reporter_country = pd.read_sql_query(sql, conn)


,REPORTER_COUNTRY,primaryid_count
0,AE,29
1,AR,348
2,AT,84
3,AU,183
4,BA,4
...,...,...
78,US,5683
79,UY,2
80,UZ,1
81,VN,12


In [20]:
# https://www.cnblogs.com/pine007/p/16904126.html
def get_con_by_country_code(country):
    if country.upper() in [
        'AX', 'AL', 'AD', 'AT', 'BY', 'BE', 'BA', 'BG', 'HR', 'CZ', 'DK', 'EE', 'FO', 'FI', 'FR', 'DE', 'GI', 'GR', 'GG', 'VA', 'HU', 'IS', 'IE', 'IM', 'IT', 'JE', 'LV', 'LI', 'LT', 'LU', 'MK', 'MT', 'MD', 'MC', 'ME', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SM', 'RS', 'SK', 'SI', 'ES', 'SJ', 'SE', 'CH', 'UA', 'GB']:
        return 'Europe'
    elif country.upper() in ['AF', 'AM', 'AZ', 'BH', 'BD', 'BT', 'BN', 'KH', 'CN', 'CC', 'CY', 'GE', 'HK', 'IN', 'ID', 'IR', 'IQ', 'IL', 'JP', 'JO', 'KZ', 'KP', 'KR', 'KW', 'KG', 'LA', 'LB', 'MO', 'MY', 'MV', 'MN', 'MM', 'NP', 'OM', 'PK', 'PS', 'PH', 'QA', 'SA', 'SG', 'LK', 'SY', 'TW', 'TJ', 'TH', 'TL', 'TR', 'TM', 'AE', 'UZ', 'VN', 'YE']:
        return 'Asia/Japan'
    elif country.upper() in ['DZ', 'AO', 'BJ', 'BW', 'IO', 'BF', 'BI', 'CM', 'CV', 'CF', 'TD', 'KM', 'CG', 'CD', 'CI', 'DJ', 'EG', 'GQ', 'ER', 'ET', 'TF', 'GA', 'GM', 'GH', 'GN', 'GW', 'KE', 'LS', 'LR', 'LY', 'MG', 'MW', 'ML', 'MR', 'MU', 'YT', 'MA', 'MZ', 'NA', 'NE', 'NG', 'RE', 'RW', 'SH', 'ST', 'SN', 'SC', 'SL', 'SO', 'ZA', 'SD', 'SZ', 'TZ', 'TG', 'TN', 'UG', 'EH', 'ZM', 'ZW']:
        return 'Africa'
    elif country.upper() in ['AI', 'AG', 'AW', 'BS', 'BB', 'BZ', 'BM', 'CA', 'KY', 'CR', 'CU', 'DM', 'DO', 'SV', 'GL', 'GD', 'GP', 'GT', 'HT', 'HN', 'JM', 'MQ', 'MX', 'MS', 'AN', 'NI', 'PA', 'PR', 'KN', 'LCA', 'PM', 'VC', 'TTO', 'TC', 'US', 'VG', 'VI']:
        return 'North America'
    elif country.upper() in ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'FK', 'GF', 'GY', 'PY', 'PE', 'SR', 'UY', 'VE']:
        return 'South America'
    elif country.upper() in ['AS', 'AU', 'CX', 'CK', 'FJ', 'PF', 'GU', 'KI', 'MH', 'FM', 'NR', 'NC', 'NZ', 'NU', 'NF', 'MP', 'PW', 'PG', 'PN', 'WS', 'SB', 'TK', 'TO', 'TV', 'UM', 'VU', 'WF']:
        return 'Oceania'
    elif country.upper() in ['BV', 'HM', 'GS']:
        return 'Antarctica'
    
    return 'Missing'
        
df_reporter_country['continent'] = df_reporter_country['REPORTER_COUNTRY'].map(get_con_by_country_code)
df_reporter_country.groupby(['continent',]).sum('primaryid_count')

,primaryid_count
continent,
Africa,339
Asia/Japan,1749
Europe,3050
Missing,2101
North America,6371
Oceania,183
South America,1532


## Time of onset 

In [ ]:
## drug usage show

In [28]:
## DROP TABLE  if exists faers.drug_usage_with_ae;

sql ='''
DROP TABLE  if exists faers.drug_usage_with_ae;
CREATE TABLE  faers.drug_usage_with_ae  as
SELECT 
    D.CASEID,
    D.PRIMARYID,
    R.DRUG_SEQ,
    coalesce(T.START_DT,'000000') as START_DT,
    D.EVENT_DT,
    T.PRIMARYID as ther_primaryid,
    T.DSG_DRUG_SEQ as ther_DSG_DRUG_SEQ
FROM (  -- caseid 下取最后一次primaryid
        select CASEID, PRIMARYID, max(EVENT_DT) as EVENT_DT
        from faers.demo_dedup
        where rn = 1
        group by 1,2
) D JOIN faers.ribo_drug_usage R    -- 该份报告记录了 ribocilicib
    on  D.PRIMARYID = R.PRIMARYID        
left JOIN faers.faers_ther T ON T.PRIMARYID = R.PRIMARYID
     and T.DSG_DRUG_SEQ = R.DRUG_SEQ    
-- WHERE T.START_DT IS NOT NULL  AND D.EVENT_DT IS NOT NULL;
'''

df_time_onset = pd.read_sql_query(sql, conn)
df_time_onset

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/1702952119.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_time_onset = pd.read_sql_query(sql, conn)


TypeError: 'NoneType' object is not iterable

In [34]:
sql = '''
    
    select 
    if(START_DT IS NOT NULL AND EVENT_DT IS NOT NULL,
        TIMESTAMPDIFF(DAY, START_DT, EVENT_DT),
        'Unknown'
    ) as time_of_onset,
    count(distinct primaryid) as primaryid_count
    from faers.drug_usage_with_ae
    group by 1 order by 1
'''
df_time_onset = pd.read_sql_query(sql, conn)
df_time_onset

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/1389168013.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_time_onset = pd.read_sql_query(sql, conn)


,time_of_onset,primaryid_count
0,None,3170
1,-1,12
2,-10,8
3,-102,1
4,-105,1
...,...,...
1031,99,5
1032,990,2
1033,994,1
1034,995,1


# Table1.7上报年份

In [38]:
##  REPT_DT 存在异常数据, 否则取：INIT_FDA_DT
sql =''' 
    SELECT
        YEAR(if(length(D.REPT_DT)<4, D.INIT_FDA_DT,D.REPT_DT)) AS report_year,
        COUNT(distinct D.PRIMARYID) AS report_count
    FROM
        faers.demo_dedup D
    JOIN 
        faers.ribo_drug_usage R
    ON D.rn = 1 and D.PRIMARYID = R.PRIMARYID
    GROUP BY
        1
    ORDER BY
        1;
'''

df_rpt_year = pd.read_sql_query(sql, conn)
df_rpt_year

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_38268/2273248669.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rpt_year = pd.read_sql_query(sql, conn)


,report_year,report_count
0,2017,473
1,2018,1161
2,2019,1575
3,2020,1490
4,2021,2222
5,2022,2958
6,2023,4547
7,2024,899


# Table2 适应症及同用药品

## 联用药物

In [47]:
# 同一个primaryid下
sql = '''

    SELECT 
        concat(d1.prod_ai) as drug_info,
        COUNT(d1.CASEID) AS concomitant_medications_caseid_count,
        COUNT(d1.primaryid) AS concomitant_medications_primaryid_count
    FROM 
        faers.faers_drug d1
    join faers.ribo_drug_usage  d2
    on d1.primaryid = d2.primaryid and  d1.caseid = d2.caseid
    where lower(d1.role_cod) != 'ps' and d1.drug_seq != d2.drug_seq
    group by 1
    ;
'''
df_concomitant = pd.read_sql_query(sql, conn)
df_concomitant

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_61625/1735953897.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_concomitant = pd.read_sql_query(sql, conn)


,drug_info,concomitant_medications_caseid_count,concomitant_medications_primaryid_count
0,RIBOCICLIB,15446,15446
1,LETROZOLE,6319,6319
2,PREGABALIN,141,141
3,IBUPROFEN,200,200
4,METHADONE HYDROCHLORIDE,18,18
...,...,...,...
1226,FENFLURAMINE HYDROCHLORIDE,1,1
1227,INDAPAMIDEPERINDOPRIL ERBUMINE,1,1
1228,ITOPRIDE,1,1
1229,DICLOFENAC SODIUMTRAMADOL HYDROCHLORIDE,1,1


In [46]:
len(df_concomitant)

3566

## 适应症

In [63]:
# 同一个primaryid下
# todo: INDI_PT 格式较为混乱，需要继续处理
sql = '''

    SELECT 
        concat(trim(d1.INDI_PT) as INDI_PT,
        COUNT(d1.CASEID) AS INDI_PT_caseid_count,
        COUNT(d1.primaryid) AS INDI_PT_primaryid_count
    FROM 
        faers.faers_indi d1
    join faers.ribo_drug_usage  d2
    on d1.primaryid = d2.primaryid 
        and d1.INDI_DRUG_SEQ = d2.drug_seq
    group by 1
    order by 1
    ;
'''
df_indi = pd.read_sql_query(sql, conn)
df_indi

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_61625/507644697.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_indi = pd.read_sql_query(sql, conn)


DatabaseError: Execution failed on sql '

    SELECT 
        concat(trim(d1.INDI_PT, '\r') as INDI_PT,
        COUNT(d1.CASEID) AS INDI_PT_caseid_count,
        COUNT(d1.primaryid) AS INDI_PT_primaryid_count
    FROM 
        faers.faers_indi d1
    join faers.ribo_drug_usage  d2
    on d1.primaryid = d2.primaryid 
        and d1.INDI_DRUG_SEQ = d2.drug_seq
    group by 1
    order by 1
    ;
': (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ', '\\r') as INDI_PT,\n        COUNT(d1.CASEID) AS INDI_PT_caseid_count,\n        CO' at line 2")

In [54]:
sum(df_indi['concomitant_medications_primaryid_count'])

19801

#  Table 3,4,5 ROR分析数据准备

## 导出 和 ribociclib 相关的 pt 

In [25]:
# 去除异常字符
sql = '''   
   SELECT 
        replace(coalesce(t2.pt,'not_found_pt'),'\\r', '') as pt,
        COUNT(distinct t1.CASEID) AS caseid_cnt,
        COUNT(distinct t1.PRIMARYID) AS primaryid_cnt
    FROM faers.ribo_drug_usage t1
    left join faers.faers_reac t2
    on t1.primaryid = t2.primaryid and lower(t1.role_cod) = 'ps'
    group by 1
    order by 1;
'''
df_pt_stat = pd.read_sql_query(sql, conn)

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_24540/1334817465.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pt_stat = pd.read_sql_query(sql, conn)


KeyError: False

In [31]:
len(df_pt_stat.pt)


3223

## 并查找其 smq 和 pt

In [37]:
df_pt_stat.to_csv('ribo_pt.csv', index=False)


## 获取相关pt 到 smq & soc的映射

In [34]:
import os
import sys
import requests
from cachetools import cached, TTLCache
import time
import pandas as pd
import tqdm
g_token=''

@cached(cache=TTLCache(maxsize=1, ttl=36000))
def get_token():
    # step 1:  using username='30007'  password='2A4nFXhP' on https://mid.meddra.org/account/register for apikey
    # step 2:  apply for grant
    cmd = '''
    curl \
        --request POST \
        --data "grant_type=password"  \
        --data "username={username}"  --data "password={apikey}" \
        --data "scope=meddraapi" \
        --data "client_id=mspclient" \
        "https://mid.meddra.org/connect/token"
    '''.format(username='30007', apikey='4Lj68iPSy3jjti7hK2hM')
    os.system(cmd)
    # ref: https://cloud.tencent.com/developer/article/1908468
    return auth['access_token']
    # {"access_token":"6acb65aaaa2eee65530de8e2521606b10051a5b8a5451bdf1227eb704bda080d","expires_in":36000,"token_type":"Bearer"}

def get_token2():
    return "0ee4891cc555c08327623cf1e37697aca59bba27744b0006e138d2f0e6a47fc7"

def get_soc(row):
    ## (1) search pt name for pt-code
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer {}'.format(get_token2()),
        'Content-Type': 'text/json',
    }

    pt_name = row['pt']   # "heart valve operation"

    data = '{"bview":"SOC","rsview":"release","language":"English","stype":1,"addlangs":[],"filters":[],"version":24,"searchterms":[{"searchtype":0,"searchterm":"' \
           + pt_name + '","searchlogic":0}],"kana":false,"idiacritical":true,"synonym":false,"contains":false,"soc":false,"hlgt":false,"hlt":false,"pt":true,"llt":false,"smq":false,"skip":0,"take":0,"separator":2}'
    
    try:
        response = requests.post('https://gxpmapiprod.meddra.org/api/search', headers=headers, data=data)
        #print(response.json())
        ##(2) detail: search soc name & code using pt_code
        pt_code = response.json()[0]['pcode']
    
        data = '{"bview":"SOC","rsview":"release","code":' + str(pt_code) + \
               ',"pcode":0,"syncode":0,"lltcode":0,"ptcode":0,"hltcode":0,"hlgtcode":0,"soccode":0,"smqcode":0,"type":"PT","addlangs":[],"rtype":"P","lang":"English","ver":24,"kana":false,"separator":2}'
    
        response = requests.post('https://gxpmapiprod.meddra.org/api/detail', headers=headers, data=data)
        soc_name = ''
        soc_code = -1
    
        for item in response.json()['mds']:
            #print(item)
            if item['primarysocfg'] == 'Y':
                soc_name = item['socname']
                soc_code = item['soccode']
                           
        ## finally output results using dataframes
        #return {'pt': pt_name, 'pt_code': pt_code, 'soc_name': soc_name, 'soc_code': soc_code}
        return pd.Series([pt_name, pt_code, soc_name, soc_code], index=['pt_name', 'pt_code', 'soc_name', 'soc_code'])

    except Exception as e:
        print("error for" + pt_name)
    finally:
        pass


In [23]:
get_token()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0

{"access_token":"0ee4891cc555c08327623cf1e37697aca59bba27744b0006e138d2f0e6a47fc7","expires_in":36000,"token_type":"Bearer"}

100   224    0   124  100   100     65     52  0:00:01  0:00:01 --:--:--   117


NameError: name 'auth' is not defined

In [35]:
import tqdm
from tqdm import tqdm
tqdm.pandas(desc='apply')
pt_soc_mapping = df_pt_stat.progress_apply(get_soc, axis=1)


apply:   1%|          | 39/3223 [01:29<2:02:27,  2.31s/it]


KeyboardInterrupt: 

In [98]:
pt_soc_mapping


,pt_name,pt_code,soc_name,soc_code
0,abdominal adhesions,10000050,Gastrointestinal disorders,10017947
1,abdominal discomfort,10000059,Gastrointestinal disorders,10017947
2,abdominal distension,10000060,Gastrointestinal disorders,10017947
3,abdominal hernia,10060954,Gastrointestinal disorders,10017947
4,abdominal hernia infection,10077232,Infections and infestations,10021881


In [ ]:
# 一个primaryid 
# check select count(1),count(distinct caseid),count(distinct primaryid) from faers_reac;

sql = '''

    SELECT 
        concat(d1.INDI_PT) as INDI_PT,
        COUNT(d1.CASEID) AS INDI_PT_caseid_count,
        COUNT(d1.primaryid) AS INDI_PT_primaryid_count
    FROM 
        faers.faers_reac d1
    join faers.ribo_drug_usage  d2
    on d1.primaryid = d2.primaryid 
        and d1.INDI_DRUG_SEQ = d2.drug_seq
    group by 1
    order by 1
    ;
'''
df_indi = pd.read_sql_query(sql, conn)
df_indi

In [ ]:
# faers.faers_reac:    PRIMARYID,CASEID,PT, SOC_NAME

## 计算ror矩阵中的 a,b,c,d

In [21]:
ALTER table faers.ror_pt_a_b  modify drug varchar(255) character set utf8mb3 collate utf8mb3_bin;

0

In [ ]:
sql = '''   
      DROP TABLE  if exists faers.ror_stats;
      CREATE TABLE faers.ror_stats as
      select t1.drug, t1.pt, 
             t1.drug_event_cnt,  
             coalesce(t2.drug_cnt,0) - t1.drug_event_cnt as drug_otherevent_cnt
      from faers.ror_pt_a_b t1
      left join faers.ror_pt_D_d t2
      on t1.drug = t2.drug
      ;
'''
df_ror_stats = pd.read_sql_query(sql, conn)

In [ ]:
sql = '''   
      DROP TABLE  if exists faers.ror_stats;
      CREATE TABLE faers.ror_stats as
      select t1.drug, t1.pt, 
             t1.drug_event_cnt,  
             coalesce(t2.drug_cnt,0) - t1.drug_event_cnt as drug_otherevent_cnt
      from faers.ror_pt_a_b t1
      left join faers.ror_pt_D_d t2
      on t1.drug = t2.drug
      ;
'''
df_ror_stats = pd.read_sql_query(sql, conn)

In [ ]:
## generation: pt, a,b,c,d
sql = '''   
      DROP TABLE  if exists faers.ror_pt_stats;
      CREATE TABLE faers.ror_pt_stats as
      select t1.drug, t1.pt, 
             t1.drug_event_cnt,  
             coalesce(t2.drug_cnt,0) - t1.drug_event_cnt as drug_otherevent_cnt
      from faers.ror_pt_a_b t1
      left join faers.ror_pt_D_d t2
      on t1.drug = t2.drug
      ;
'''
df_ror_pt_stats = pd.read_sql_query(sql, conn)

In [ ]:
## generation: pt, a,b,c,d
sql = '''   
      DROP TABLE  if exists faers.ror_pt_abcd;
      CREATE TABLE faers.ror_pt_abcd as
      
      select pt, drug_event_cnt as DE, 0 as dE,  drug_otherevent_cnt as De, 0 as de
          from faers.ror_pt_stats where drug = 'D'
          
      union all 
    
      select pt, 0 as DE, drug_event_cnt as dE, 0  as De, drug_otherevent_cnt as de
          from faers.ror_pt_stats where drug = 'd'
        
      ;
'''
df_ror_pt_abcd = pd.read_sql_query(sql, conn)

# TABLE 3,4,5 指标计算

## python指标算子

In [40]:
import math
# 定义相关指标的计算 python函数
## https://www.jianshu.com/p/69dfbcceef9b
def get_ROR(row):
    import math
    a = row['DE']
    b = row['De']
    c = row['dE']
    d = row['de']    
    ror = a/c/(b/d)
    ci_lower = math.log(ror) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ci_upper = math.log(ror) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    return a/c/(b/d), math.exp(ci_lower), math.exp(ci_upper)

def get_PRR(row):

    a = row['DE']
    b = row['De']
    c = row['dE']
    d = row['de']    
    prr = a/(a+b)/(c/(c+d))
    ci_lower = math.log(prr) - 1.96 * math.sqrt(1/a - 1/(a+b) + 1/c - 1/(c+d))
    ci_upper = math.log(prr) + 1.96 * math.sqrt(1/a - 1/(a+b) + 1/c - 1/(c+d))
    return prr, math.exp(ci_lower), math.exp(ci_upper)

def get_MHRA(row):

    a = row['DE']
    b = row['De']
    c = row['dE']
    d = row['de']    
    
    chi_squared = math.pow( (a*d - b*c), 2) * (a+b+c+d) / ((a+b)*(c+d)*(a+c)*(b+d))
    return chi_squared


def get_EBGM(row):

    a = row['DE']
    b = row['De']
    c = row['dE']
    d = row['de']    
    
    EBGM = a*(a+b+c+d)/(a+b)/(a+c)
    ci_lower = math.log(EBGM) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ci_upper = math.log(EBGM) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    
    return EBGM,  math.exp(ci_lower), math.exp(ci_upper)


def get_BCPNN(row):
    

In [70]:
## reads: pt, a,b,c,d
sql = '''   
      select * from faers.ror_pt_abcd where a>0;
'''
df_ror_pt_abcd = pd.read_sql_query(sql, conn)


/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_24540/305480428.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ror_pt_abcd = pd.read_sql_query(sql, conn)


In [42]:
df_ror_pt_abcd.to_excel('pt_abcd.xlsx', index=False)

NameError: name 'get_ROR' is not defined

In [45]:
df_ror_pt_abcd.head()

,pt,a,b,c,d
0,abdominal adhesions,2.0,1912.0,20157.0,11855172.0
1,abdominal discomfort,555.0,114630.0,19604.0,11742454.0
2,abdominal distension,216.0,60888.0,19943.0,11796196.0
3,abdominal hernia,6.0,3160.0,20153.0,11853924.0
4,abdominal hernia infection,1.0,36.0,20158.0,11857048.0


In [49]:
len(df_ror_pt_abcd)

3223

In [90]:
import math
# 定义相关指标的计算 python函数
## https://www.jianshu.com/p/69dfbcceef9b
def get_ROR(row):
    import math

    a = row['a']
    b = row['b']
    c = row['c']
    d = row['d']
    
    if a*b*c*d == 0:
        return -1,-1,-1
    
    ror = a/c/(b/d)
    ci_lower = math.log(ror) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ci_upper = math.log(ror) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    return a/c/(b/d), math.exp(ci_lower), math.exp(ci_upper)

def get_PRR(row):

    a = row['a']
    b = row['b']
    c = row['c']
    d = row['d']    
    
    if a * (a+b) * (c+d) * (b+d) * (a+c) * c == 0:
        return -1,-1,-1,-1
    
    
    prr = a/(a+b)/(c/(c+d))
    ci_lower = math.log(prr) - 1.96 * math.sqrt(1/a - 1/(a+b) + 1/c - 1/(c+d))
    ci_upper = math.log(prr) + 1.96 * math.sqrt(1/a - 1/(a+b) + 1/c - 1/(c+d))
    chi_squared = math.pow( (a*d - b*c), 2) * (a+b+c+d) / ((a+b)*(c+d)*(a+c)*(b+d))  # get_MHRA

    return prr, math.exp(ci_lower), math.exp(ci_upper),chi_squared

def get_BCPNN(row):

    a = row['a']
    b = row['b']
    c = row['c']
    d = row['d'] 
    
    C = a+b+c+d
    Cx = a + b
    Cy = a + c
    Cxy = a
    
    α1 = 1
    β1 = 1
    α = 2
    β = 1
    γ11 = 1
    
    if True: 
        γ = γ11 * (C + α) * ( C + β) / (Cx + α1) / (Cy + β1)        
        IC = math.log2(a*(a+b+c+d)*(a+c)*(a+b))
        E_IC = math.log2( (Cxy+γ11) * (C+α) * (C+β) / (C+γ) / (Cx+α1) / (Cy+β1) )
        V_IC = ( (C-Cxy+γ-γ11)/(Cxy+γ11)/(1+C+γ) + (C-Cx+α-α1)/(Cx+α1)/(1+C+α) + (C-Cy+β-β1)/(Cy+β1)/(1+C+β) ) / (math.log(2) * math.log(2))
        CI_lower = E_IC - 2 * math.sqrt(V_IC)
        CI_upper = E_IC + 2 * math.sqrt(V_IC)
               
        return IC, E_IC, V_IC, CI_lower, CI_upper
    
    #except Exception as e:
    #    print('error for {}'.format(row))
    #    return -1,-1,-1,-1,-1
    #finally:
    #    pass

def get_MPGS(row):

    a = row['a']
    b = row['b']
    c = row['c']
    d = row['d']    
    
    if a*b*c*d*(a+c)*(a+b) == 0:
        return -1,-1,-1
    
    EBGM = a*(a+b+c+d)/(a+c)/(a+b)
    ci_lower = math.log(EBGM) - 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    ci_upper = math.log(EBGM) + 1.96 * math.sqrt(1/a + 1/b + 1/c + 1/d)
    
    return EBGM,  math.exp(ci_lower), math.exp(ci_upper)


sql = '''   
      select * from faers.ror_pt_abcd where a>0;
'''
df_ror_pt_abcd = pd.read_sql_query(sql, conn)


df_ror_pt_abcd[["ROR", "ROR_95%CI_lower", "ROR_95%CI_upper"]] =  df_ror_pt_abcd.apply(get_ROR, axis = 1, result_type = "expand")
df_ror_pt_abcd[["PRR", "PRR_95%CI_lower", "PRR_95%CI_upper", "χ2"]] =  df_ror_pt_abcd.apply(get_PRR, axis = 1, result_type = "expand")
df_ror_pt_abcd[["BCPNN_IC","BCPNN_E(IC)", "BCPNN_V(IC)", "BCPNN_CI_lower", "BCPNN_CI_upper"]] =  df_ror_pt_abcd.apply(get_BCPNN, axis = 1, result_type = "expand")
df_ror_pt_abcd[["EBGM", "EBGM_CI_lower", "EBGM_CI_upper"]] =  df_ror_pt_abcd.apply(get_MPGS, axis = 1, result_type = "expand")
df_ror_pt_abcd.to_excel('ror_result.xlsx', index=False)
os.system('open ror_result.xlsx')

/var/folders/7w/sgw5xk457ss20939d0n2_5600000gn/T/ipykernel_24540/3978857261.py:95: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ror_pt_abcd = pd.read_sql_query(sql, conn)


0